In [1]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]]), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
1,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
2,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
3,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
4,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
...,...,...,...
12128,Und in Sands bewege sich ja einiges .,Und in Sands,sich1573
12129,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
12130,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573
12131,"Am Anfang gab es viel Widerstand , man holpert...","Am Anfang gab es viel Widerstand , man holperte",sich1573


In [3]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
...,...,...,...
1573,Und in Sands bewege sich ja einiges .,Und in Sands,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573


In [4]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if row['cue'] == '':  # if the string in cue is empty
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0]]
    # if len(output[0].split()) > 300:  # 1024 is the max token length of the later model, let's take roughly a third of that as a max of words
    #     output = [" ".join(output[0].split()[-300:]), output[1]]
    # if len(output[1].split()) > 300:
    #     output = [output[0], " ".join(output[1].split()[-300:])]
    return output

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
...,...,...,...
1573,Und in Sands bewege sich ja einiges .,Und in Sands,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573


In [5]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...  
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...  
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...  
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...  

[562 rows x 2 columns]

In [6]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
677,"Bei der Konstruktion ""Additiv_Koordinativkompo..."
563,"Bei der ""Gegenüberstellung:V2_V2-Konstruktion""..."
696,"Die ""Konjunktiv:V.conj-I-Konstruktion"" gehört ..."
488,"Die ""Superlativ:am_meisten/wenigstenADJ-Konstr..."
674,"Die ""Kausaler_Konnektor:weil-Konstruktion"" die..."
...,...
875,"Bei der Konstruktion (Kxn) ""Kategorisierung_Tr..."
1323,Die meist als Prädikativum verwendete Konstruk...
85,"Die ""Reduplikation_Quantifizierung:N1_über_N1-..."
1792,"Bei der Konstruktion (Kxn) ""Prädikation_Negati..."


In [7]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)  # TODO: Aussortierte Definitions betrachten!
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...   
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...   
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...   
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[562 rows x 3 columns]

##### Generieren neuer Sätze:

In [23]:
import numpy as np

pseudowords = np.load("../../out/pseudowords_comapp_133_281.npy")
pseudowords

array([[ 0.00146393, -0.05308372,  0.01719784, ..., -0.03633863,
        -0.04242704,  0.05942685],
       [-0.00954064, -0.10843118, -0.04376286, ...,  0.02453525,
         0.01159196,  0.07159752],
       [ 0.04643255,  0.0565825 ,  0.09630286, ...,  0.05671702,
         0.11930588, -0.16660306],
       ...,
       [ 0.0018719 , -0.15932393, -0.01882566, ..., -0.01438776,
        -0.20695591,  0.17447297],
       [ 0.05854554,  0.02364353, -0.0815459 , ...,  0.04878142,
        -0.05361592, -0.10101417],
       [ 0.01017702, -0.04054715, -0.05801943, ..., -0.04664138,
         0.01116951,  0.00059162]])

In [9]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    json_data = json.load(json_file)
json_data

[{'label': 'am488',
  'target1': 'Doch der theatralische Selbstmord von General Slobodan Praljak , der direkt dem Urteilsspruch folgte , entwertete all jene Tode , die weder öffentlich noch theatralisch waren , und am wenigsten freiwillig , sondern sich auf den Bauernhöfen , in den Kellern der Familienhäuser und in den in Lager umgewandelten Warenmagazine abspielten .',
  'target1_idx': 28,
  'query': 'Doch der theatralische Selbstmord von General Slobodan Praljak , der direkt dem Urteilsspruch folgte , entwertete all <mask> , die weder öffentlich noch theatralisch waren , und am wenigsten freiwillig , sondern sich auf den Bauernhöfen , in den Kellern der Familienhäuser und in den in Lager umgewandelten Warenmagazine abspielten .',
  'query_idx': 27},
 {'label': 'wenigsten488',
  'target1': 'Doch der theatralische Selbstmord von General Slobodan Praljak , der direkt dem Urteilsspruch folgte , entwertete all jene Tode , die weder öffentlich noch theatralisch waren , und am wenigsten fre

In [10]:
json_data.sort(key=lambda x: x["label"])  # Grouping doesn't work without sorting first!
json_data = [list(group) for _, group in itertools.groupby(json_data, key=lambda x: x["label"])]
mbart_tokens = [d[0]['label'] for d in json_data]  # TODO Check whether the labels are in correct order!

mbart_tokens, len(mbart_tokens)

(['""Was13',
  '"647',
  '"Wir-äh-spielen-äh-in-der-äh-Champions-League647',
  '(1597',
  '(1600',
  '(1602',
  '(1637',
  '(1641',
  '(1643',
  '(379',
  '(579',
  '(581',
  '(584',
  '(590',
  '(592',
  '(600',
  '(886',
  '(892',
  '(900',
  '(905',
  '(907',
  '(909',
  '(917',
  '(919',
  '(921',
  ')1597',
  ')1600',
  ')1602',
  ')1637',
  ')1641',
  ')1643',
  ')1792',
  ')379',
  ')579',
  ')581',
  ')584',
  ')590',
  ')592',
  ')600',
  ')886',
  ')892',
  ')900',
  ')905',
  ')907',
  ')909',
  ')917',
  ')919',
  ')921',
  ')«579',
  ',1459',
  ',973',
  '-128',
  '-651',
  '-654',
  '-875',
  '-973',
  ':595',
  ':875',
  ':973',
  'Abstand683',
  'Allein20',
  'Aller1630',
  'Als1315',
  'Als133',
  'Als1770',
  'Am488',
  'Am492',
  'Am500',
  'Amerika605',
  'Anstatt320',
  'Art129',
  'Arzt1509',
  'Augenblick1301',
  'Ausmaß1777',
  'BRUTAL1503',
  'Besser1762',
  'Bis559',
  'Brutal1503',
  'Buche1346',
  'Das1313',
  'Das1461',
  'Dass21',
  'Dasselbe104',
  'Der13

Load the vanilla mBART-50 model:

In [11]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50", return_dict=True)
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE")
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Add to existing embeddings:

In [12]:
combined_embeddings = torch.cat((model.model.encoder.embed_tokens.weight, torch.tensor(pseudowords)), dim=0)
model.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.model.encoder.embed_tokens

Embedding(250327, 1024)

Add to existing tokens:

In [13]:
tokenizer.add_tokens(mbart_tokens[-len(pseudowords):])
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 250327. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(250327, 1024)

Complete the cues:

In [22]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            input_text = cue + " " + row["pseudoword"]
            
            assert row["pseudoword"] in tokenizer.tokenize(input_text)  # TODO Entfernen sobald alle Pseudowords da sind
            # print(tokenizer.tokenize(input_text))
            
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            outputs = tokenizer(input_text, return_tensors="pt")
            outputs = model.generate(outputs["input_ids"], max_length=target_length, num_return_sequences=1,
                                     num_beams=10, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        print(row["pseudoword"], str(output_texts), str([float(score) for score in scores]))
        
        return pd.DataFrame([str(output_texts), str([float(score) for score in scores])])
    except Exception as e:  # TODO Entfernen, sobald alle Pseudowords da sind
        print(".", end="")
        return pd.DataFrame([row["pseudoword"], str(e), "[-1.0]"])

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["example", "cue", "pseudoword"]].progress_apply(complete_cues, axis=1)  # TODO use "pseudoword" as index
pseudoword_output_scores

  0%|          | 0/562 [00:00<?, ?it/s]

...


KeyboardInterrupt



In [ ]:
examples[["pseudoword", "generated", "score"]] = pseudoword_output_scores

examples

#### Anzahl neuer Sätze

In [ ]:
examples.to_csv(f"../../out/comapp/data.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/data.xlsx")